In [1]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline, DistilBertTokenizer, DistilBertForSequenceClassification
import torch

In [8]:
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
tokenizer = AutoTokenizer.from_pretrained("martin-ha/toxic-comment-model")
toxic_classifier = AutoModelForSequenceClassification.from_pretrained("martin-ha/toxic-comment-model")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Keyword arguments {'model': 'j-hartmann/emotion-english-distilroberta-base'} not recognized.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-bas

In [9]:
inputs = tokenizer("I love you!", return_tensors="pt")
with torch.no_grad():
    logits = distilbert(**inputs).logits
predicted_class_id = logits.argmax().item()
distilbert.config.id2label[predicted_class_id]

'LABEL_1'

In [10]:
emotion_classifier("I hate this!")

TypeError: 'list' object is not callable

In [6]:
pipeline =  TextClassificationPipeline(model=toxic_classifier, tokenizer=tokenizer)
print(pipeline('Sucks to suck'))

[{'label': 'toxic', 'score': 0.9524264931678772}]
